In [3]:
import requests
import time

In [8]:
# Dados dos Tokens (endereços e decimais)
TOKENS = {
    "BRZ": {"address": "0x4eD141110F6EeeAbA9A1df36d8c26f684d2475Dc", "decimals": 18},
    "USDT": {"address": "0xc2132d05d31c914a87c6611c10748aeb04b58e8f", "decimals": 6},
    "BRLA": {"address": "0xe6a537a407488807f0bbeb0038b79004f19dddfb", "decimals": 18},
    "POL": {"address": "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE", "decimals": 18},
}

In [9]:
# Função para consultar preços de swap
def consultar_preco(src_token, dest_token, amount):
    if TOKENS[src_token]["decimals"] == 6:  # Exemplo: USDT
        amount = int(amount * 100) / 100.0  # Arredonda para baixo

    params = {
        "srcToken": TOKENS[src_token]["address"],
        "destToken": TOKENS[dest_token]["address"],
        "amount": str(int(amount * 10 ** TOKENS[src_token]["decimals"])),
        "network": "137",  # Rede Polygon
        "side": "SELL",  # Vender token de origem
        "srcDecimals": TOKENS[src_token]["decimals"],
        "destDecimals": TOKENS[dest_token]["decimals"],
    }

    response = requests.get("https://apiv5.paraswap.io/prices", params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Erro: {response.status_code}", "details": response.json()}


In [10]:
# Função para processar dados de swap
def parse_swap_data(data, src_token, dest_token):
    if "priceRoute" not in data:
        return {"error": "Dados inválidos ou incompletos"}

    price_route = data["priceRoute"]
    src_amount = int(price_route["srcAmount"]) / (10 ** TOKENS[src_token]["decimals"])
    dest_amount = int(price_route["destAmount"]) / (10 ** TOKENS[dest_token]["decimals"])

    swap_fee_percent = 0.0015  # Taxa de Swap: 0.15%
    swap_fee = dest_amount * swap_fee_percent
    dest_after_fee = dest_amount - swap_fee

    return {
        "src_amount": float(src_amount),
        "dest_amount_after_fee": float(dest_after_fee),
    }

In [12]:
def retornar_para_brz(brla_amount, route):
    if route == "USDT":
        # Etapa 1: BRLA -> USDT
        brla_to_usdt_data = consultar_preco("BRLA", "USDT", brla_amount)
        if "error" in brla_to_usdt_data:
            return {"error": "Erro na conversão BRLA -> USDT", "details": brla_to_usdt_data}

        brla_to_usdt_result = parse_swap_data(brla_to_usdt_data, "BRLA", "USDT")
        usdt_received = brla_to_usdt_result["dest_amount_after_fee"]

        # Etapa 2: USDT -> BRZ
        usdt_to_brz_data = consultar_preco("USDT", "BRZ", usdt_received)
        if "error" in usdt_to_brz_data:
            return {"error": "Erro na conversão USDT -> BRZ", "details": usdt_to_brz_data}

        usdt_to_brz_result = parse_swap_data(usdt_to_brz_data, "USDT", "BRZ")
        brz_received = usdt_to_brz_result["dest_amount_after_fee"]

    elif route == "POL":
        # Etapa 1: BRLA -> POL
        brla_to_pol_data = consultar_preco("BRLA", "POL", brla_amount)
        if "error" in brla_to_pol_data:
            return {"error": "Erro na conversão BRLA -> POL", "details": brla_to_pol_data}

        brla_to_pol_result = parse_swap_data(brla_to_pol_data, "BRLA", "POL")
        pol_received = brla_to_pol_result["dest_amount_after_fee"]

        # Etapa 2: POL -> BRZ
        pol_to_brz_data = consultar_preco("POL", "BRZ", pol_received)
        if "error" in pol_to_brz_data:
            return {"error": "Erro na conversão POL -> BRZ", "details": pol_to_brz_data}

        pol_to_brz_result = parse_swap_data(pol_to_brz_data, "POL", "BRZ")
        brz_received = pol_to_brz_result["dest_amount_after_fee"]

    # Cálculo do lucro ou prejuízo no retorno
    return {
        "Rota de Retorno": route,
        "BRLA inicial": brla_amount,
        "BRZ final": brz_received,
        "Lucro no Retorno": brz_received - brla_amount,
    }

def monitorar_retorno(brla_amount, intervalo=10):
    while True:
        try:
            # Retornar via USDT
            resultado_usdt = retornar_para_brz(brla_amount, "USDT")
            print("Retorno via USDT:")
            for chave, valor in resultado_usdt.items():
                if isinstance(valor, float):  # Formatar apenas valores numéricos
                    print(f"{chave}: {valor:.6f}")
                else:
                    print(f"{chave}: {valor}")
            print("-" * 50)

            # Retornar via POL
            resultado_pol = retornar_para_brz(brla_amount, "POL")
            print("Retorno via POL:")
            for chave, valor in resultado_pol.items():
                if isinstance(valor, float):  # Formatar apenas valores numéricos
                    print(f"{chave}: {valor:.6f}")
                else:
                    print(f"{chave}: {valor}")
            print("-" * 50)

        except Exception as e:
            print(f"Erro inesperado: {e}")
        time.sleep(intervalo)


# Monitorar retorno
monitorar_retorno(brla_amount=350, intervalo=10)


Retorno via USDT:
Rota de Retorno: USDT
BRLA inicial: 350
BRZ final: 346.577326
Lucro no Retorno: -3.422674
--------------------------------------------------
Retorno via POL:
Rota de Retorno: POL
BRLA inicial: 350
BRZ final: 346.393212
Lucro no Retorno: -3.606788
--------------------------------------------------
Retorno via USDT:
Rota de Retorno: USDT
BRLA inicial: 350
BRZ final: 346.637523
Lucro no Retorno: -3.362477
--------------------------------------------------
Retorno via POL:
Rota de Retorno: POL
BRLA inicial: 350
BRZ final: 346.443598
Lucro no Retorno: -3.556402
--------------------------------------------------


KeyboardInterrupt: 